In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
pesu1=pd.read_json('pesu1_tweets.json')
pesu2=pd.read_json('pesu2_tweets.json')
pesu3=pd.read_json('pesu3_tweets.json')
pesu4=pd.read_json('pesu4_tweets.json')
pesu5=pd.read_json('pesu5_tweets.json')
pesu6=pd.read_json('pesu6_tweets.json')

In [3]:
tweets_df = pd.concat([pesu1,pesu2,pesu3,pesu4,pesu5,pesu6], axis=0)
tweets_df = tweets_df.loc[:, ['text' , 'username' ,'tweet_id', 'timestamp' ,'hashtags' , 'screen_name','retweets']]
tweets_df.reset_index()
tweets_df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets
0,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2
1,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0
2,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2
3,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0
4,Talk to us for instant updates on direct admis...,universitiesbangalore.com,1139797459436261376,2019-06-15 07:31:05,"[Direct_Admission_PES_University, PES_Universi...",universitiesba1,0
5,Y is the service Road from PES university in 1...,Abhilash BP,1139483889603248128,2019-06-14 10:45:04,[],bp_abhilash,0
6,Two Day FDP on DESIGN FOR TEST using MENTOR TE...,CoreEL Technologies,1139429982025814016,2019-06-14 07:10:51,[],CoreEL_Techno,1
7,@AddlCPTraffic @blrcitytraffic service rd nea...,Shreedhar Kulkarni,1139072421917384704,2019-06-13 07:30:02,[],ShreedharKulka7,0
8,PES University is calling applications for the...,Lawctopus,1139044540780670976,2019-06-13 05:39:15,[],Lawctopus,0
9,Talk to us for instant updates on direct admis...,universitiesbangalore.com,1139797459436261376,2019-06-15 07:31:05,"[Direct_Admission_PES_University, PES_Universi...",universitiesba1,0


In [9]:
tweets_df = tweets_df[~tweets_df.index.duplicated()]
uni_tweets = tweets_df.loc[tweets_df['screen_name'].isin(['PESUniversity','dscpesu','pesu_io'])]

In [10]:
uni_tweets['uni_handle'] = 1
uni_tweets

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,uni_handle
0,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1
1,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1
2,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1
3,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1
19,"ATTEND LAW INFO SESSION on July 14, 10 am. Giv...",PES University,1149686503284473856,2019-07-12 14:26:37,[],PESUniversity,0,1
21,Talk on Human Robot Interaction by Prof. David...,PES University,1149673086406447104,2019-07-12 13:33:18,[],PESUniversity,0,1
24,"ATTEND LAW INFO SESSION on July 14, 10 am. Giv...",PES University,1149686503284473856,2019-07-12 14:26:37,[],PESUniversity,0,1
26,Talk on Human Robot Interaction by Prof. David...,PES University,1149673086406447104,2019-07-12 13:33:18,[],PESUniversity,0,1
28,Dr. Snehanshu Saha wins PEACE Award-Distinguis...,PES University,1150797222058942464,2019-07-15 16:00:13,[],PESUniversity,0,1
29,Dr. Snehanshu Saha wins PEACE Award-Distinguis...,PES University,1150797222058942464,2019-07-15 16:00:13,[],PESUniversity,0,1


In [11]:
tweets_df = tweets_df[~tweets_df.index.duplicated()]
tweets_df['uni_handle'] = uni_tweets['uni_handle']
pesu = tweets_df.combine_first(tweets_df)
pesu

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,uni_handle
0,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1.0
1,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1.0
2,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1.0
3,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1.0
4,Talk to us for instant updates on direct admis...,universitiesbangalore.com,1139797459436261376,2019-06-15 07:31:05,"[Direct_Admission_PES_University, PES_Universi...",universitiesba1,0,NaN
5,Y is the service Road from PES university in 1...,Abhilash BP,1139483889603248128,2019-06-14 10:45:04,[],bp_abhilash,0,NaN
6,Two Day FDP on DESIGN FOR TEST using MENTOR TE...,CoreEL Technologies,1139429982025814016,2019-06-14 07:10:51,[],CoreEL_Techno,1,NaN
7,@AddlCPTraffic @blrcitytraffic service rd nea...,Shreedhar Kulkarni,1139072421917384704,2019-06-13 07:30:02,[],ShreedharKulka7,0,NaN
8,PES University is calling applications for the...,Lawctopus,1139044540780670976,2019-06-13 05:39:15,[],Lawctopus,0,NaN
9,Talk to us for instant updates on direct admis...,universitiesbangalore.com,1139797459436261376,2019-06-15 07:31:05,"[Direct_Admission_PES_University, PES_Universi...",universitiesba1,0,NaN


In [12]:
pesu[pesu['uni_handle'] == 1]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,uni_handle
0,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1.0
1,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1.0
2,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1.0
3,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1.0
19,"ATTEND LAW INFO SESSION on July 14, 10 am. Giv...",PES University,1149686503284473856,2019-07-12 14:26:37,[],PESUniversity,0,1.0
21,Talk on Human Robot Interaction by Prof. David...,PES University,1149673086406447104,2019-07-12 13:33:18,[],PESUniversity,0,1.0
24,"ATTEND LAW INFO SESSION on July 14, 10 am. Giv...",PES University,1149686503284473856,2019-07-12 14:26:37,[],PESUniversity,0,1.0
26,Talk on Human Robot Interaction by Prof. David...,PES University,1149673086406447104,2019-07-12 13:33:18,[],PESUniversity,0,1.0
28,Dr. Snehanshu Saha wins PEACE Award-Distinguis...,PES University,1150797222058942464,2019-07-15 16:00:13,[],PESUniversity,0,1.0
29,Dr. Snehanshu Saha wins PEACE Award-Distinguis...,PES University,1150797222058942464,2019-07-15 16:00:13,[],PESUniversity,0,1.0


In [13]:
pesu['uni_handle'] = pesu['uni_handle'].replace(np.nan, 0)
pesu[pesu['uni_handle'] == 1]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,uni_handle
0,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1.0
1,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1.0
2,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1.0
3,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1.0
19,"ATTEND LAW INFO SESSION on July 14, 10 am. Giv...",PES University,1149686503284473856,2019-07-12 14:26:37,[],PESUniversity,0,1.0
21,Talk on Human Robot Interaction by Prof. David...,PES University,1149673086406447104,2019-07-12 13:33:18,[],PESUniversity,0,1.0
24,"ATTEND LAW INFO SESSION on July 14, 10 am. Giv...",PES University,1149686503284473856,2019-07-12 14:26:37,[],PESUniversity,0,1.0
26,Talk on Human Robot Interaction by Prof. David...,PES University,1149673086406447104,2019-07-12 13:33:18,[],PESUniversity,0,1.0
28,Dr. Snehanshu Saha wins PEACE Award-Distinguis...,PES University,1150797222058942464,2019-07-15 16:00:13,[],PESUniversity,0,1.0
29,Dr. Snehanshu Saha wins PEACE Award-Distinguis...,PES University,1150797222058942464,2019-07-15 16:00:13,[],PESUniversity,0,1.0


In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
pesu['compound'] = [analyzer.polarity_scores(v)['compound'] for v in pesu['text']]

In [16]:
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

pesu['Sentiment'] = pesu['compound'].apply(lambda x: sentiment(x))

In [17]:
pesu

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,uni_handle,compound,Sentiment
0,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1.0,0.5106,Positive
1,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1.0,0.4215,Positive
2,Transforming footpaths: The architecture depar...,PES University,1137730835044392960,2019-06-09 14:39:03,[],PESUniversity,2,1.0,0.5106,Positive
3,MBA OPEN HOUSE on June 9 @ 9.30 am. You and yo...,PES University,1137231356377780224,2019-06-08 05:34:18,[],PESUniversity,0,1.0,0.4215,Positive
4,Talk to us for instant updates on direct admis...,universitiesbangalore.com,1139797459436261376,2019-06-15 07:31:05,"[Direct_Admission_PES_University, PES_Universi...",universitiesba1,0,0.0,0.0000,Neutral
5,Y is the service Road from PES university in 1...,Abhilash BP,1139483889603248128,2019-06-14 10:45:04,[],bp_abhilash,0,0.0,0.3612,Positive
6,Two Day FDP on DESIGN FOR TEST using MENTOR TE...,CoreEL Technologies,1139429982025814016,2019-06-14 07:10:51,[],CoreEL_Techno,1,0.0,0.0000,Neutral
7,@AddlCPTraffic @blrcitytraffic service rd nea...,Shreedhar Kulkarni,1139072421917384704,2019-06-13 07:30:02,[],ShreedharKulka7,0,0.0,-0.3753,Negative
8,PES University is calling applications for the...,Lawctopus,1139044540780670976,2019-06-13 05:39:15,[],Lawctopus,0,0.0,0.0000,Neutral
9,Talk to us for instant updates on direct admis...,universitiesbangalore.com,1139797459436261376,2019-06-15 07:31:05,"[Direct_Admission_PES_University, PES_Universi...",universitiesba1,0,0.0,0.0000,Neutral


In [19]:
pesu.to_csv('final_pesu.csv')